In [19]:
import pandas as pd
import numpy as np

a=pd.read_csv('../data/dict_links.csv',index_col=0)
dict_links=a.to_dict(orient='dict')
dict_links=dict_links['links']

dict_links['UNIFOND RENTABILIDAD OBJETIVO IV, FI']

'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V04981494 '

In [67]:
import requests
import json
import csv
from bs4 import BeautifulSoup
import os

infile = open("../data/DOC (20).XML","r")
contents = infile.read()
soup = BeautifulSoup(contents, 'lxml')

# bla=requests.get('https://www.cnmv.es/portal/Consultas/wuc/DescargaXBRLIPP.ashx?t={2a2d5290-274d-43f0-93db-dbef7a5a184d}')
# soup = BeautifulSoup(bla.content, 'lxml')

# soup


In [ ]:
fondo='MISTRAL CARTERA EQUILIBRADA, FI'
def inspect_data(fondo):
     fondo_df=pd.read_csv(f'../data/csv/{fondo}.csv', sep='*')
     return fondo_df

fondo_df=inspect_data(fondo)
pd.set_option('display.max_rows', len(fondo_df))
pd.set_option('display.max_columns', len(fondo_df.columns))
fondo_df

In [77]:
clases = soup.find_all('iic-com:denominacionclase')
clases

[]

In [14]:
dict_links['AMUNDI CORTO PLAZO, FI']

'https://www.cnmv.es/portal/Consultas/IIC/Fondo.aspx?nif=V80346265 '

In [43]:
def parse_strings(x):
    try: return float(x)
    except: return None

In [44]:
for i in ['rentab_IIC_trim',	'volat_vl_trim',	'ratio_gastos_trim']:
    df[i]=df[i].apply(parse_strings)

                      

In [96]:
str(df[df['rentab_avg'].isnull()].rentab_avg[1117])


'nan'

In [ ]:
filt=df['fondo']=='BANKIA GESTION DE AUTOR, FI'

a=df[filt]
a.sort_values('period')

In [1]:
import os
import numpy as np
import pandas as pd

def create_df():
    df=False
    i=0
    for file in os.listdir('../data/csv'):
#         print(file, 'parsed')
        if type(df)==bool:
            df=pd.read_csv(f'../data/csv/{file}', sep='*')
        else:
            df=df.append(pd.read_csv(f'../data/csv/{file}', sep='*'))
        i+=1
    print(f'{i} files parsed!')
    return df.drop_duplicates()
df=create_df()    

1042 files parsed!


In [41]:
def wrang_main(df):

    #converting vars
    df['start_date'] = pd.to_datetime(df['start_date'])
    df['end_date'] = pd.to_datetime(df['end_date'])

    #filling nulls
    df['riesgo'].fillna('ND', inplace=True)
    df['rentab_avg'].fillna(df['remun_liq'], inplace=True)

    #cleaning data
    df['dividendos']=df['dividendos'].apply(div_clean)
    for i in ['beneficio','rentab_IIC_trim', 'volat_vl_trim', 'ratio_gastos_trim',
              'comision_gest_pat','comision_gest_res','comision_gest_total',
              'comision_depos']:
        df[i] = df[i].apply(parse_strings)

    df['period_type'] = df.apply(lambda x: period_type(x['period']), axis=1)
    df['year'] = df.apply(lambda x: x['start_date'].year, axis=1)
    #
    #yearly data
    #export calculated data csv

    #new cols

    return df

In [3]:
def period_type(x):
    if 'S' in x:
        return 'S'
    elif 'T' in x:
        return 'T'
    elif 'Y' in x:
        return Y

def div_clean(x):
    if type(x)==str:
        if x.strip().lower()=='false' or x.strip().lower()=='[]':
            return False
        if x.strip().lower()== 'true':
            return True
    if type(x)==bool:
        return x

def parse_strings(x):
    try: return float(x)
    except: return None

In [4]:
def yearly_data(S1,S2):
    array=[]
    mean_vars=['rotacion','rentab_avg','remun_liq','n_participaciones',
               'n_participes', ]
    sum_vars=['beneficio','comision_gest_pat','comision_gest_res','comision_gest_total',
              'comision_depos']
    copy_vars=['fondo','name', 'end_date','registro_CNMV', 'patrimonio', 'valor_liq','riesgo','clase']

    null_vars= ['rentab_IIC_trim',
               'volat_vl_trim','ratio_gastos_trim']

    calc_vars=['period', 'start_date','dividendos','period_type', 'year']

    columns=['fondo', 'name', 'period', 'start_date', 'end_date', 'riesgo',
       'rotacion', 'rentab_avg', 'remun_liq', 'n_participaciones',
       'n_participes', 'beneficio', 'patrimonio', 'dividendos', 'valor_liq',
       'comision_gest_pat', 'comision_gest_res', 'comision_gest_total',
       'comision_depos', 'rentab_IIC_trim', 'volat_vl_trim',
       'ratio_gastos_trim', 'clase', 'period_type', 'year']

    for i in range(len(columns)):
        variable=columns[i]
        if variable in mean_vars:
            array.append((S1[variable]+S2[variable])/2)
        elif variable in sum_vars:
            array.append(S1[variable]+S2[variable])
        elif variable in copy_vars:
            array.append(S2[variable])
        elif variable in calc_vars:
            if variable=='period':
                array.append(f'{S1[variable][:4]} Y')
            elif variable=='start_date':
                array.append(S1[variable])
            elif variable == 'dividendos':
                array.append(S1[variable]*S2[variable])
            elif variable=='period_type':
                array.append('Y')
            elif variable=='year':
                array.append(S1.period[:4])
        elif variable in null_vars:
            array.append(None)
        
    return array

In [46]:
from tqdm import tqdm 

def create_data(df):
    df.reset_index(drop=True, inplace=True)
    df_calc = False
    for fondo in tqdm(df.fondo.unique()):
#         print(fondo)
        df_fondo=df[df.fondo==fondo]
        for year in df_fondo.year.unique():
#             print(year)
            names=[fondo]
            df_year=df_fondo[(df_fondo.year==year)]
            if df_year.clase.sum()>0:
                names=df_year.name.unique()
            for name in names:
                df_name=df_year[df_year.name==name]
                S1 = ''
                S2 = ''
                T1 = ''
                T3 = ''
                tempY=''
                tempT2=''
                tempT4=''
                try:S1=df_name.loc[df_name.period==f'{year} S1'].squeeze(axis=0)
                except:pass
                try: S2 = df_name.loc[df_name.period == f'{year} S2'].squeeze(axis=0)
                except: pass
                try:T1=df_name.loc[df_name.period==f'{year} T1'].squeeze(axis=0)
                except:pass
                try:T3=df_name.loc[df_name.period==f'{year} T3'].squeeze(axis=0)
                except:pass
#                 print(type(S1),type(S2),type(T1),type(T3))
#                 print(len(S1),len(S2),len(T1),len(T3))
                if len(S1)>0 and len(S2)>0:
                    tempY=yearly_data(S1, S2)
                if len(S1)>0 and len(T1)>0:
                    tempT2=''
                if len(T3)>0 and len(S2)>0:
                    tempT4=''

                temp_list=[tempY,tempT2,tempT4]   
#                 print(temp_list)
                for datum in temp_list:
                    if datum!='':
                        if type(df_calc)==bool:
                            col_list = ['fondo', 'name', 'period', 'start_date', 'end_date', 'riesgo',
                                        'rotacion', 'rentab_avg', 'remun_liq', 'n_participaciones',
                                        'n_participes', 'beneficio', 'patrimonio', 'dividendos', 'valor_liq',
                                        'comision_gest_pat', 'comision_gest_res', 'comision_gest_total',
                                        'comision_depos', 'rentab_IIC_trim', 'volat_vl_trim',
                                        'ratio_gastos_trim', 'clase', 'period_type', 'year']
                            df_calc = pd.DataFrame(datum,index=col_list).T
                            #df_calc.columns = col_list
                        else:
                            df_calc.loc[len(df_calc)] = yearly_data(S1, S2)
    return df_calc

In [42]:
df=wrang_main(df)

In [47]:
try1=create_data(df)

100%|██████████| 1042/1042 [01:48<00:00,  9.63it/s]


In [ ]:
df[df.fondo=='1 KESSLER GLOBAL, FI'&]

In [52]:
try1.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4502 entries, 0 to 4501
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   fondo                4502 non-null   object        
 1   name                 4502 non-null   object        
 2   period               4502 non-null   object        
 3   start_date           4502 non-null   datetime64[ns]
 4   end_date             4502 non-null   datetime64[ns]
 5   riesgo               4502 non-null   object        
 6   rotacion             4500 non-null   object        
 7   rentab_avg           4502 non-null   object        
 8   remun_liq            0 non-null      object        
 9   n_participaciones    4502 non-null   object        
 10  n_participes         4502 non-null   object        
 11  beneficio            3868 non-null   object        
 12  patrimonio           4502 non-null   object        
 13  dividendos           4502 non-nul

In [13]:
df.head()

,fondo,name,period,start_date,end_date,riesgo,rotacion,rentab_avg,remun_liq,n_participaciones,...,comision_gest_pat,comision_gest_res,comision_gest_total,comision_depos,rentab_IIC_trim,volat_vl_trim,ratio_gastos_trim,clase,period_type,year
0,"GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI","I2 DESARROLLO SOSTENIBLE ISR, FI",2016 S2,2016-07-01,2016-12-31,4,0.00,0.0,NaN,177692.59,...,0.43,0,0.43,0.03,-1.05,0.98,0.74,False,S,2016
1,"GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI","I2 DESARROLLO SOSTENIBLE ISR, FI",2016 T3,2016-07-01,2016-09-30,4 en una escala del 1 al 7,0.00,0.0,NaN,30000.00,...,0.08,0,0.08,0.00,NaN,NaN,0.33,False,T,2016
2,"GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI","I2 DESARROLLO SOSTENIBLE ISR, FI",2017 S2,2017-07-01,2017-12-31,4 en una escala del 1 al 7.,0.01,0.0,NaN,300738.48,...,0.68,0,0.68,0.05,-1.19,1.73,0.41,False,S,2017
3,"GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI","I2 DESARROLLO SOSTENIBLE ISR, FI",2017 T3,2017-07-01,2017-09-30,4 en una escala del 1 al 7,0.00,0.0,NaN,301569.24,...,0.34,0,0.34,0.03,-0.59,1.75,0.42,False,T,2017
4,"GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI","I2 DESARROLLO SOSTENIBLE ISR, FI",2017 S1,2017-01-01,2017-06-30,4 en una escala del 1 al 7,0.19,0.0,NaN,229008.41,...,0.67,0,0.67,0.05,-0.31,2.61,0.43,False,S,2017


In [22]:
def T2_data(T1,S1):
    array=[]
    mean_vars=['rotacion','rentab_avg','remun_liq','n_participaciones',
               'n_participes', ]
    sum_vars=['beneficio','comision_gest_pat','comision_gest_res','comision_gest_total',
              'comision_depos']
    copy_vars=['fondo','name', 'end_date','registro_CNMV', 'patrimonio', 'valor_liq','riesgo','clase']

    null_vars= ['rentab_IIC_trim',
               'volat_vl_trim','ratio_gastos_trim']

    calc_vars=['period', 'start_date','dividendos','period_type', 'year']

    columns=['fondo', 'name', 'period', 'start_date', 'end_date', 'riesgo',
       'rotacion', 'rentab_avg', 'remun_liq', 'n_participaciones',
       'n_participes', 'beneficio', 'patrimonio', 'dividendos', 'valor_liq',
       'comision_gest_pat', 'comision_gest_res', 'comision_gest_total',
       'comision_depos', 'rentab_IIC_trim', 'volat_vl_trim',
       'ratio_gastos_trim', 'clase', 'period_type', 'year']

    for i in range(len(columns)):
        variable=columns[i]
        if variable in mean_vars:
            array.append((2*S1[i]-T1[i]))
        elif variable in sum_vars:
            array.append(S1[i]-T1[i])
        elif variable in copy_vars:
            array.append(S1[i])
        elif variable in calc_vars:
            if variable=='period':
                array.append(f'{S1[i][:4]}')
            elif variable=='start_date':
                array.append(T1[i])
            elif variable == 'dividendos':
                array.append(bool(T1[i]+S1[i]))
            elif variable=='period_type':
                array.append('Y')
            elif variable=='year':
                array.append(S1[2][:4])
        elif variable in null_vars:
            array.append(None)

    return array

fondo                  GESIURIS I2 DESARROLLO SOSTENIBLE ISR, FI
name                            I2 DESARROLLO SOSTENIBLE ISR, FI
period                                                   2016 S2
start_date                                   2016-07-01 00:00:00
end_date                                     2016-12-31 00:00:00
riesgo                                                         4
rotacion                                                       0
rentab_avg                                                     0
remun_liq                                                    NaN
n_participaciones                                         177693
n_participes                                                   9
beneficio                                                      0
patrimonio                                           1.73774e+06
dividendos                                                 False
valor_liq                                                 9.7795
comision_gest_pat        

In [27]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44691 entries, 0 to 44690
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   fondo                44691 non-null  object        
 1   name                 44691 non-null  object        
 2   period               44691 non-null  object        
 3   start_date           44691 non-null  datetime64[ns]
 4   end_date             44691 non-null  datetime64[ns]
 5   riesgo               44691 non-null  object        
 6   rotacion             44591 non-null  float64       
 7   rentab_avg           44593 non-null  float64       
 8   remun_liq            89 non-null     float64       
 9   n_participaciones    44593 non-null  float64       
 10  n_participes         44593 non-null  float64       
 11  beneficio            43931 non-null  object        
 12  patrimonio           44593 non-null  float64       
 13  dividendos           44593 non-

In [28]:
    sum_vars=['beneficio','comision_gest_pat','comision_gest_res','comision_gest_total',
              'comision_depos']

In [40]:
def type_checker(x,t):
    return type(x)!=t
    
        
df[df['comision_gest_pat'].apply(lambda x: type_checker(x,t=float))].comision_gest_pat.unique()

array(['[]', '0.06', '0.23', '0.12', '0.44', '0.28', '0.13', '0.35',
       '0.45', '0.22', '0.2', '0.18', '0.1', '0.09', '0.14', '0.11',
       '0.02', '0.17', '0.48', '0.0', '0.95', '0.47', '0.96', '0.94',
       '0.46', '0.25', '0.49', '0.5', '0.51', '0.26', '0.24', '0.16',
       '0.07', '0.33', '0.34', '0.32', '0.08', '0.55', '1.09', '1.11',
       '0.56', '0.54', '1.1', '0.43', '0.21', '0.42', '0.4', '0.27',
       '0.57', '1.12', '0.92', '1.14', '0.97', '0.89', '1.13', '0.91',
       '0.9', '0.38', '0.19', '0.37', '0.52', '0.53', '0.62', '0.98',
       '0.88', '0.31', '0.63', '0.75', '0.77', '0.39', '0.76', '0.74',
       '1.0', '0.84', '1.01', '0.7', '0.99', '0.85', '0.69', '0.81',
       '0.71', '0.05', '0.04', '0.03', '0.93', '0.36', '0.72', '1.05',
       '1.04', '0.78', '0.41', '0.68', '0.3', '0.01', '0.87', '0.86',
       '1.02', '0.67', '0.73', '0.6', '0.65', '0.15', '0.29', '0.59',
       '0.61', '1.06', '1.07', '0.66', '1.51',
       '[<iic-com:comisiongestioncobradapat

In [67]:
def yearly_data(S1,S2):
    array=[]
    mean_vars=['rotacion','rentab_avg','remun_liq','n_participaciones',
               'n_participes', ]
    sum_vars=['beneficio','comision_gest_pat','comision_gest_res','comision_gest_total',
              'comision_depos']
    copy_vars=['fondo','name', 'end_date','registro_CNMV', 'patrimonio', 'valor_liq','riesgo','clase']

    null_vars= ['rentab_IIC_trim',
               'volat_vl_trim','ratio_gastos_trim']

    calc_vars=['period', 'start_date','dividendos', 'period_type', 'year']

    columns=['fondo', 'name', 'period', 'start_date', 'end_date', 'riesgo',
       'rotacion', 'rentab_avg', 'remun_liq', 'n_participaciones',
       'n_participes', 'beneficio', 'patrimonio', 'dividendos', 'valor_liq',
       'comision_gest_pat', 'comision_gest_res', 'comision_gest_total',
       'comision_depos', 'rentab_IIC_trim', 'volat_vl_trim',
       'ratio_gastos_trim', 'clase','period_type', 'year']

    for i in range(len(columns)):
        variable=columns[i]
        if variable in mean_vars:
            array.append((S1[i]+S2[i])/2)
        elif variable in sum_vars:
            array.append(S1[i]+S2[i])
        elif variable in copy_vars:
            array.append(S2[i])
        elif variable in calc_vars:
            if variable=='period':
                array.append(f'{S1[i][:4]} Y')
            elif variable=='start_date':
                array.append(S1[i])
            elif variable == 'dividendos':
                array.append(bool(S1[i]+S2[i]))
            elif variable=='period_type':
                array.append(S1[2][-2])
            elif variable=='year':
                array.append(S1[2][:4])
        elif variable in null_vars:
            array.append(None)

    return array

In [68]:
df.reset_index(drop=True,inplace=True)
S1=df.iloc[3,:]
S2=df.iloc[1,:]
x=yearly_data(S1,S2)
x

['AZVALOR IBERIA, FI',
 'AZVALOR IBERIA FI',
 '2016 Y',
 '2016-01-01',
 '2016-12-31',
 '6 en una escala del 1 al 7.',
 1.09,
 0.0,
 nan,
 798254.98,
 1211.0,
 0.0,
 111412520,
 False,
 114.262,
 1.79,
 nan,
 1.79,
 0.04,
 None,
 None,
 None,
 False,
 'S',
 '2016']

In [74]:
    col_list=['fondo', 'name', 'period', 'start_date', 'end_date', 'riesgo',
       'rotacion', 'rentab_avg', 'remun_liq', 'n_participaciones',
       'n_participes', 'beneficio', 'patrimonio', 'dividendos', 'valor_liq',
       'comision_gest_pat', 'comision_gest_res', 'comision_gest_total',
       'comision_depos', 'rentab_IIC_trim', 'volat_vl_trim',
       'ratio_gastos_trim', 'clase','period_type', 'year']

df_calc=pd.DataFrame(yearly_data(S1, S2)).T
df_calc.columns=col_list
df_calc

,fondo,name,period,start_date,end_date,riesgo,rotacion,rentab_avg,remun_liq,n_participaciones,...,comision_gest_pat,comision_gest_res,comision_gest_total,comision_depos,rentab_IIC_trim,volat_vl_trim,ratio_gastos_trim,clase,period_type,year
0,"AZVALOR IBERIA, FI",AZVALOR IBERIA FI,2016 Y,2016-01-01,2016-12-31,6 en una escala del 1 al 7.,1.09,0,NaN,798255,...,1.79,NaN,1.79,0.04,None,None,None,False,S,2016


In [63]:
'2019 T1'[-2]

'T'

In [ ]:
filt1=df.fondo==ADRIZA GLOBAL, FI
filt2=df.fondo==2019 S1

df[]

In [3]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])

In [42]:
type(list(df.loc[32,'volat_vl_trim'])[0])

str

In [ ]:
df.head()

In [12]:
a=df[filt1].drop_duplicates().fondo.unique()
a.sort()
a

array(['AMUNDI CORTO PLAZO, FI', 'CAIXABANK BOLSA ALL CAPS ESPAÑA, FI',
       'CAIXABANK BOLSA GESTION EURO, FI',
       'CAIXABANK COMUNICACION MUNDIAL, FI',
       'CAIXABANK DIVERSIFICADO DINAMICO, FI',
       'CAIXABANK OPORTUNIDAD, FI',
       'CAIXABANK RENTA FIJA ALTA CALIDAD CREDITICIA, FI',
       'CAIXABANK RENTA FIJA DOLAR, FI',
       'CAIXABANK RF DURACION NEGATIVA, FI',
       'CAIXABANK SELECCION ALTERNATIVA, FI', 'CALIOPE, FI',
       'FIDEFONDO, FI', 'FONDITEL ALBATROS, FI',
       'FONDITEL BOLSA MUNDIAL SOSTENIBLE, FI',
       'FONDITEL RENTA FIJA MIXTA INTERNACIONAL, FI',
       'INVERSABADELL 10, FI', 'INVERSABADELL 25, FI',
       'INVERSABADELL 50, FI', 'INVERSABADELL 70, FI',
       'SABADELL AMERICA LATINA BOLSA, FI',
       'SABADELL ASIA EMERGENTE BOLSA, FI',
       'SABADELL BONOS ALTO INTERES, FI', 'SABADELL BONOS EMERGENTES, FI',
       'SABADELL BONOS ESPAÑA, FI', 'SABADELL BONOS EURO, FI',
       'SABADELL BONOS FLOTANTES EURO, F.I.',
       'SABADELL B

In [41]:
df['rating_depos'].fillna('ND',inplace=True)

In [48]:
df.dividendos.unique()
for i in df.dividendos.unique():
    print(i)
    print(type(i))

None
<class 'NoneType'>
False
<class 'bool'>


In [81]:
df.sort_values(['fondo','period'],inplace=True)
df.reset_index(drop=True,inplace=True)
df[19:25]

,level_0,index,fondo,name,period,start_date,end_date,riesgo,rotacion,rentab_avg,...,patrimonio,dividendos,valor_liq,comision_gest_pat,comision_gest_res,comision_gest_total,comision_depos,rentab_IIC_trim,volat_vl_trim,ratio_gastos_trim
19,19,21,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 S1,2019-01-01,2019-06-30,7 (En una escala del 1 al 7),0.04,-0.30,...,7396175,False,8.6995,0.5,0,0.50,0.04,-1.71,6.81,0.39
20,20,19,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 S2,2019-07-01,2019-12-31,7 (En una escala del 1 al 7),0.00,-0.40,...,3453739,False,7.9879,0.51,0,0.51,0.04,-8.07,9.46,0.43
21,21,22,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 T1,2019-01-01,2019-03-31,7 (En una escala del 1 al 7),0.00,-0.30,...,7544242,False,8.8508,0.25,0,0.25,0.02,-4.17,4.34,0.4
22,22,20,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 T3,2019-07-01,2019-09-30,7 (En una escala del 1 al 7),0.00,-0.30,...,5642385,False,8.6890,0.25,0,0.25,0.02,-0.12,11.83,0.46
23,23,1,"A&G RENTA FIJA CORTO PLAZO, FI","AYG TESORERIA, FI",2009 S1,2009-01-01,2009-06-30,Moderado,2.09,1.34,...,90722000,False,5.1817,NaN,NaN,0.12,0.04,0.34,0.12,0.08
24,24,0,"A&G RENTA FIJA CORTO PLAZO, FI","AYG TESORERIA, FI",2009 T3,2009-07-01,2009-09-30,Moderado,0.00,0.22,...,78034000,False,5.1949,NaN,NaN,0.06,0.02,0.25,0.08,0.08


In [90]:
df[df['period']>='2019 S1']

,level_0,index,fondo,name,period,start_date,end_date,riesgo,rotacion,rentab_avg,...,patrimonio,dividendos,valor_liq,comision_gest_pat,comision_gest_res,comision_gest_total,comision_depos,rentab_IIC_trim,volat_vl_trim,ratio_gastos_trim
19,19,21,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 S1,2019-01-01,2019-06-30,7 (En una escala del 1 al 7),0.04,-0.30,...,7396175,False,8.6995,0.5,0,0.50,0.04,-1.71,6.81,0.39
20,20,19,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 S2,2019-07-01,2019-12-31,7 (En una escala del 1 al 7),0.00,-0.40,...,3453739,False,7.9879,0.51,0,0.51,0.04,-8.07,9.46,0.43
21,21,22,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 T1,2019-01-01,2019-03-31,7 (En una escala del 1 al 7),0.00,-0.30,...,7544242,False,8.8508,0.25,0,0.25,0.02,-4.17,4.34,0.4
22,22,20,"1 KESSLER GLOBAL, FI",1 KESSLER GLOBAL FI,2019 T3,2019-07-01,2019-09-30,7 (En una escala del 1 al 7),0.00,-0.30,...,5642385,False,8.6890,0.25,0,0.25,0.02,-0.12,11.83,0.46
59,59,38,"A&G RENTA FIJA CORTO PLAZO, FI","AYG RENTA FIJA CORTO PLAZO, FI",2019 S1,2019-01-01,2019-06-30,Moderado,0.11,-0.30,...,26911180,False,5.5473,0.07,NaN,0.07,0.04,-0.06,0.08,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9705,9705,43,"BBVA GESTION CONSERVADORA, FI","BBVA GESTION CONSERVADORA, FI",2020 T1,2020-01-01,2020-03-31,3 en una escala del 1 al 7,0.57,-0.36,...,470008000,False,9.9014,0.5,0,0.50,0.05,-7.41,12.04,0.58
9706,9706,41,"BBVA GESTION CONSERVADORA, FI","BBVA GESTION CONSERVADORA, FI",2020 T3,2020-07-01,2020-09-30,3 en una escala del 1 al 7,0.44,-0.12,...,475690000,False,10.1618,0.5,0,0.50,0.05,0.1,2.88,0.6
9707,9707,0,"ÁNCORA CONSERVADOR, FI",CLASE I ANCORA CONSERVADOR FI,2020 S 2,2020-07-01,2020-12-31,3 en una escala de 1 a 7,0.00,-0.29,...,3035946,False,101.1982,0.07,0,0.07,0.01,1.2,1.33,0.18
9708,9708,2,"ÁNCORA CONSERVADOR, FI",CLASE R ANCORA CONSERVADOR FI,2020 T 3,2020-07-01,2020-09-30,3 en una escala de 1 a 7,0.00,-0.17,...,11559780,False,100.2478,0.14,0,0.14,0.02,[],[],0.35


In [9]:
#New columns
def period_type(x):
    if 'S' in x:
        return 'S'
    elif 'T' in x:
        return 'T'
    
def trimester_4(s2,t3):
    return 2*s2 - t3

def trimester_2(s1,t1):
    return 2*s1 - t1

In [10]:
df['period_type']=df['period'].apply(period_type)

In [27]:
fondo='ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO'
os.remove(f'data/csv/{fondo}.csv')
print(f'{fondo}.csv deleted')

ACCION EUROSTOXX 50 ETF, FI COTIZADO ARMONIZADO.csv deleted
